# This file is for creating the test accuracies of the varying models

In [1]:
# Imports
import sys
import pandas as pd
import numpy as np
from rfgap import RFGAP


# Get the data
sys.path.insert(0, '/yunity/arusty/PF-GAP')

static2024 = pd.read_csv('../data/static2024.csv')
static2023 = pd.read_csv('../data/static2023.csv')
time_series = pd.read_csv('../data/time_series.csv')
labels = pd.read_csv('../data/labels.csv')
labels = np.array(labels).flatten()


from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test, static2023_train, static2023_test, static2024_train, static2024_test = train_test_split(time_series, 
labels, static2023, static2024, test_size=0.2, random_state=42)


# X_train = np.concat([X_train, static2023_train], axis=1)
# X_test = np.concat([X_test, static2024_test], axis=1)


# RF - Rocket

In [ ]:
from RFGAP_Rocket.RFGAP_Rocket import RFGAP_Rocket
rocket = RFGAP_Rocket(prediction_type = "classification", rocket = "Multi",
                         n_kernels=512)
rocket.fit(X_train, y_train, static2023_train, weights = None)
pred = rocket.predict(X_test, static2024_test)

score = (pred == y_test).mean()
print(f"Accuracy: {score:.2f}")
